In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing 
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer 

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(train_df.shape)
print(test_df.shape)

(7613, 5)
(3263, 4)


In [26]:
#what does the data look like

train_df.iloc[:1,:]
clense_data=True

In [27]:
# what do disaster tweets looke like, juat check it loaded fine

disasters = train_df[train_df["target"]==1]
disasters.iloc[:2,:]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [28]:
#clense data is so instructed, to test whether cleaning data improves performance
#lets clean the data
if (clense_data==True):
    %run DataClenser.ipynb
    train_df['text'] = train_df['text'].apply(lambda x: text_clense(x))
    test_df['text'] = test_df['text'].apply(lambda x: text_clense(x))

tää on emiserkki: https://dillindallin.com
Huba haba
What shall we do today zequestion
correct me please
haa hee ja jee  


In [29]:
#combine data 
df=pd.concat([train_df,test_df])
df.shape

/Users/marttiylikoski/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(10876, 5)

In [30]:
%run DataHelpers.ipynb
embedding_dict = get_pretrained_glove_vectors('./glove.6B/glove.6B.100d.txt')
corpus=create_corpus_new(df)
#count_vectorizer = feature_extraction.text.CountVectorizer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marttiylikoski/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 10876/10876 [00:01<00:00, 8726.74it/s]


In [31]:
#keras imports
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

In [32]:

MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))
print (word_index['hiring'])
#word_index.keys()

Number of unique words: 22807
981


In [33]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    if i < num_words:
        emb_vec=embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i]=emb_vec 

100%|██████████| 22807/22807 [00:00<00:00, 519984.62it/s]


In [34]:
tweet_pad[0][0:]
#tweet_pad.shape


array([ 100, 5778,   20,    1,  814,    5,   18,  237,  136, 1902, 3703,
         68,   39,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [35]:
#baseline model
model=Sequential()

embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=3e-4)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           2280800   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 50, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 2,361,301
Trainable params: 80,501
Non-trainable params: 2,280,800
_________________________________________________________________


In [37]:
train=tweet_pad[:train_df.shape[0]]
test=tweet_pad[train_df.shape[0]:]

In [52]:
X_train,X_test,y_train,y_test=train_test_split(train,train_df['target'].values,test_size=0.2)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (6090, 50)
Shape of Validation  (1523, 50)


In [53]:
# Recomended 10-20 epochs
history=model.fit(X_train,y_train,batch_size=4,epochs=10,validation_data=(X_test,y_test),verbose=2)

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
 - 31s - loss: 0.5833 - accuracy: 0.7199 - val_loss: 0.5233 - val_accuracy: 0.7748
Epoch 2/10
 - 29s - loss: 0.5507 - accuracy: 0.7476 - val_loss: 0.4740 - val_accuracy: 0.7951
Epoch 3/10
 - 29s - loss: 0.5193 - accuracy: 0.7662 - val_loss: 0.4781 - val_accuracy: 0.7945
Epoch 4/10
 - 29s - loss: 0.5130 - accuracy: 0.7660 - val_loss: 0.4631 - val_accuracy: 0.8011
Epoch 5/10
 - 29s - loss: 0.5103 - accuracy: 0.7764 - val_loss: 0.4755 - val_accuracy: 0.7978
Epoch 6/10
 - 29s - loss: 0.5054 - accuracy: 0.7706 - val_loss: 0.4538 - val_accuracy: 0.7991
Epoch 7/10
 - 1846s - loss: 0.4936 - accuracy: 0.7788 - val_loss: 0.4418 - val_accuracy: 0.7984
Epoch 8/10
 - 7479s - loss: 0.4904 - accuracy: 0.7821 - val_loss: 0.4450 - val_accuracy: 0.8076
Epoch 9/10
 - 105s - loss: 0.4763 - accuracy: 0.7852 - val_loss: 0.4428 - val_accuracy: 0.7978
Epoch 10/10
 - 68s - loss: 0.4757 - accuracy: 0.7911 - val_loss: 0.4296 - val_accuracy: 0.8011


In [56]:
train_pred_GloVe = model.predict(train)
train_pred_GloVe_int = train_pred_GloVe.round().astype('int')
train_pred_GloVe_int.shape

(7613, 1)

In [59]:
test_pred_GloVe = model.predict(test)
test_pred_GloVe_int = test_pred_GloVe.round().astype('int')
test_pred_GloVe_int.shape

(3263, 1)

In [60]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = test_pred_GloVe_int
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [62]:
sample_submission.to_csv("SubmissionGlove.csv", index=False)
